# Group by all-mathces-all-principals instead of a second lookup
We're looking for overlap of governors in companies we already have listed. We don't care about other companies (eg. that could be located in Spokane, not Seattle) so we don't need a second look up to the corps and charities database. Instead, we can make clusters based on the output of all-matches-principals.csv, where there is a row for each governor for each company in our list. 

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re
import geopandas as gp
import urllib.parse

In [2]:
df = pd.read_csv("all_matches_principals.csv", index_col=[0])
df.index.name = "index"
df.head()

,SearchTerm,BusinessName,UBINumber,BusinessId,Address,Status,address_match,ubi_match,id_match,isMatch,Agent,EntityType,PrincipalID,PrincipalName
index,,,,,,,,,,,,,,
0,KAR STANDARD LLC,KAR STANDARD LLC,604 145 518,763141,"1 FEDERAL ST FL 17, BOSTON, MA, 02110-2003, UN...",Active,False,False,False,1.0,CORPORATION SERVICE COMPANY,Entity,1570323.0,KAONOULU RANCH LLLP
1,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,604 670 512,1399595,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",Active,False,False,False,1.0,NaN,Individual,3315049.0,JOHN M. GREELEY
1,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,604 670 512,1399595,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",Active,False,False,False,1.0,NaN,Individual,3349317.0,MARTIN SELIG
2,MSI - 1ST & KING LLC,MSI - 1ST & KING LLC,602 739 680,880019,"316 OCCIDENTAL AVE S, STE 300, SEATTLE, WA, 98...",Active,False,False,False,1.0,FIKSO KRETSCHMER SMITH DIXON ORMSETH PS,Individual,2114613.0,"H MARTIN SMITH, III"
3,BRICKMAN PACIFIC LLC,BRICKMAN PACIFIC LLC,603 445 367,74763,"C/O BRICKMAN, ONCE GREENWICH OFFICE PARK, BUIL...",Active,False,False,False,1.0,C T CORPORATION SYSTEM,Entity,3617104.0,BRICKMAN FUND VI REIT INC.


In [3]:
df.value_counts(subset=["BusinessName"])

BusinessName                                
BPP 800 FIFTH PROPERTY OWNER LLC                82
BPP 999 THIRD AVENUE OWNER LLC                  81
BPP 1420 FIFTH AVENUE OWNER LLC                 78
EXPEDITORS INTERNATIONAL OF WASHINGTON, INC.    52
TMT OLIVE LAB, INC.                             18
                                                ..
HINES NORTON SEATTLE LLC                         1
HOWARD BUILDING SEATTLE, LLC                     1
HUDSON 1099 STEWART STREET, LLC                  1
HUDSON 1918 EIGHTH AVENUE, LLC                   1
WRI 2200 WESTLAKE LP                             1
Length: 235, dtype: int64

In [4]:
# Given a principal name, get all of the companies that share this principal
def get_all_companies_from_principal_dataframe(principal_match_list, principal_name):
    # Get all of the business names that have this principal
    # then return that subset
    # If NaN Principals? 
    companies_with_same_principal = principal_match_list[principal_match_list['PrincipalName'] == principal_name].BusinessId.unique()

    # if("PLYMOUTH HOUSING GROUP" in (companies_with_same_principal)): print("PLYMOUTH")
    return principal_match_list[principal_match_list["BusinessId"].isin(companies_with_same_principal)]


In [5]:
def group_companies_by_principal(principal_match_list):
    columns=['SearchTerm', 'BusinessName', 'PotentialRelatedCompany', 'UBINumber', 'BusinessId', 'Address', 'Status', 'Agent', 'Principals', 'isMatch', 'notes']
    results = pd.DataFrame([], columns)
    for idx, row in principal_match_list.iterrows():
        possible_matching_companies_df = get_all_companies_from_principal_dataframe(principal_match_list, row['PrincipalName'])
        grouped = possible_matching_companies_df.groupby("BusinessName")
        for name, group in grouped:
            principals_list = group["PrincipalName"].tolist()
            principals_list.sort()
            poss_company = group.iloc[0]

            # # row['BusinessName']: the name we mapped to from SearchTerm
            # # poss_company['BusinessName]: PotentialRelatedCompany
            new_row = pd.Series(data=[row['SearchTerm'], 
                            row["BusinessName"], 
                            poss_company["BusinessName"],
                            poss_company['UBINumber'],
                            poss_company["BusinessId"],
                            poss_company["Address"],
                            poss_company["Status"],
                            poss_company["Agent"],
                            principals_list,
                            "", # isMatch
                            ""  # Notes
                            ], 
                        index = columns)
            results = pd.concat([new_row.to_frame().T, results], ignore_index=True).drop_duplicates(subset=["UBINumber"]).dropna(how='all')
            # results = results[results['BusinessId'].isin(all_matches['BusinessId'])]
        if(idx % 25 == 0): 
            #print(f"Processing row {idx} of principal_match_list, results is {len(results)}")
            results.to_csv("companies_and_potential_matches_no_lookup.csv")
    return results

In [6]:
df_clusters = group_companies_by_principal(df)
df_lookup = pd.read_csv("companies_and_potential_matches.csv")

In [44]:
A = set(df_clusters["UBINumber"]) # no items in A that aren't in B
B = set(df_lookup["UBINumber"]) # only 1 item in B that isn't in A. it's admin dissolved, so might not matter? 
# It's from how we used to dropna- agent and entitytype and principalId are all NaN

In [47]:
in_clusters_not_lookup = A.difference(B)
in_lookup_not_clusters = B.difference(A)

In [61]:
# The companies in clusters and not lookup are those that don't share governors with anyone else in our dataset
df_clusters[df_clusters["UBINumber"].isin(in_clusters_not_lookup)]

,SearchTerm,BusinessName,PotentialRelatedCompany,UBINumber,BusinessId,Address,Status,Agent,Principals,isMatch,notes
4,8TH & PINE JOINT VENTURE,8TH & PINE JOINT VENTURE LLC,8TH & PINE JOINT VENTURE LLC,603 459 107,1092741,"10400 NE 4TH ST, #2225, BELLEVUE, WA, 98004-51...",Active,CORPORATION SERVICE COMPANY,[8TH & PINE FINANCIAL LIMITED LIABILITY COMPANY],,
5,SEATTLE HOST HOTEL COMPANY L L C,SEATTLE HOST HOTEL COMPANY LLC,SEATTLE HOST HOTEL COMPANY LLC,602 658 705,944347,"4747 BETHESDA AVENUE, SUITE 1300, BETHESDA, MD...",Active,CORPORATION SERVICE COMPANY,"[ BENJAMIN FRANKLIN HOTEL, INC, W & S REALTY ...",,
6,VANCE HOTEL ASSOCIATES LIMITED PARTNERSHIP,"VANCE HOTEL ASSOCIATES, LLC","VANCE HOTEL ASSOCIATES, LLC",602 678 924,318440,"1531 7TH AVE, SEATTLE, WA, 98101, UNITED STATES",Active,GORDON D SONDLAND,[ VANCE HOTEL HOLDINGS LLC],,
7,WORLDMARK THE CLUB,"WORLDMARK, THE CLUB","WORLDMARK, THE CLUB",601 203 364,1056986,"6277 SEA HARBOR DR, ORLANDO, FL, 32821-8027, U...",Active,CORPORATION SERVICE COMPANY,"[BOB HARTSOCK, CECILIA CUEVAS, MARK PAYETTE, M...",,
8,TODAYS HOTEL SEATTLE CORPORATION,TODAY'S HOTEL SEATTLE CORPORATION,TODAY'S HOTEL SEATTLE CORPORATION,601 877 875,1047522,"1113 6TH AVE, SEATTLE, WA, 98101-3002, UNITED ...",Active,"OMW R.A. SERVICES, LLC","[CARMEL FU, ROBERT C. NICHOLAS, VIVIAN LEE]",,
9,WIRELESS PROPCO LLC,"WIRELESS PROPCO, LLC","WIRELESS PROPCO, LLC",604 921 988,1566971,"44 S BROADWAY STE 601, WHITE PLAINS, NY, 10601...",Active,CORPORATION SERVICE COMPANY,[MELODY COMMUNICATIONS INFRASTRUCTURE II LLC],,
10,MOODY NATIONAL YALE-SEATTLE HOLDINGS LLC,"MOODY NATIONAL YALE-SEATTLE HOLDING, LLC","MOODY NATIONAL YALE-SEATTLE HOLDING, LLC",603 579 184,209933,"9655 KATY FWY STE 600, HOUSTON, TX, 77024-1385...",Active,CORPORATION SERVICE COMPANY,[MOODY NATIONAL OPERATING PARTNERSHIP I LP],,
11,MARKET PLACE CENTER LTD,"MARKET PLACE CENTER, LTD.","MARKET PLACE CENTER, LTD.",600 583 233,829762,"86 PINE ST, SEATTLE, WA, 98101-1531, UNITED ST...",Active,"BS&G, INC.","[ALI HOTEL LLC, FIRSTDOWN LLC, TALITOR LLC]",,
12,MILNER VENTURES LLC,"MILNER VENTURES, LLC","MILNER VENTURES, LLC",602 259 240,334251,"31217 NE 110TH ST, CARNATION, WA, 98014-9743, ...",Delinquent,NaN,[CHRISTINE TAPERT],,
13,1415 5TH AVENUE SEATTLE LLC,"1415 5TH AVENUE, SEATTLE, LLC","1415 5TH AVENUE, SEATTLE, LLC",604 186 121,60754,"C/O JUNSON CAPITAL, UNITS 5211-12, 52/F, THE C...",Active,C T CORPORATION SYSTEM,[JUNSON ASSETS MANAGEMENT LLC],,
